In [9]:
import pandas as pd
from pyproj import Proj, transform
from ast import literal_eval

In [4]:
incident_knmi = pd.read_csv("../dsp-dagster/data/test.csv", low_memory=False)
tree = pd.read_csv("../dsp-dagster/data/tree_data.csv", low_memory=False)

In [12]:
incident_knmi.head()

,Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,...,Incident_Starttime_Minute,Incident_Endtime_Minute,Incident_Duration_Minute,Deployment_ID,Vehicle_Type,Vehicle_Role,Fire_Station,Fire_Station_Service_Status,Driving_Time_To_Incident,Vehicle
0,240,2005-01-01,1,260,40.0,30,60,68,NaN,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,240,2005-01-01,2,230,30.0,30,60,65,NaN,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,240,2005-01-01,3,230,40.0,30,50,43,NaN,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,240,2005-01-01,4,220,40.0,40,50,38,NaN,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,240,2005-01-01,5,230,40.0,40,50,38,NaN,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
tree.head()

,_links,id,gbdBuurtId,geometrie,typeBeheerderPlus,boomhoogteklasseActueel,typeEigenaarPlus,jaarVanAanleg,soortnaam,stamdiameterklasse,standplaatsGedetailleerd,typeObject,typeSoortnaam,soortnaamKort,soortnaamTop
0,{'schema': 'https://schemas.data.amsterdam.nl/...,919933,3.630980e+12,"{'type': 'Point', 'coordinates': [122115.11, 4...",Stadsdeel Zuid,e. 15 tot 18 m.,Gemeente Amsterdam,1948.0,Tilia americana,NaN,NaN,Boom niet vrij uitgroeiend,Bomen,Tilia,Linde (Tilia)
1,{'schema': 'https://schemas.data.amsterdam.nl/...,919934,3.630980e+12,"{'type': 'Point', 'coordinates': [121980.46, 4...",Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,1978.0,Ulmus hollandica 'Vegeta',NaN,Tegels,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus)
2,{'schema': 'https://schemas.data.amsterdam.nl/...,919935,3.630980e+12,"{'type': 'Point', 'coordinates': [118737.81, 4...",Stadsdeel Zuid,c. 9 tot 12 m.,Gemeente Amsterdam,1990.0,Fraxinus excelsior 'Westhof's Glorie',"0,2 tot 0,3 m.",NaN,Boom niet vrij uitgroeiend,Bomen,Fraxinus,Es (Fraxinus)
3,{'schema': 'https://schemas.data.amsterdam.nl/...,919936,3.630980e+12,"{'type': 'Point', 'coordinates': [122053.62, 4...",Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,2002.0,Ulmus glabra 'Lutescens',NaN,NaN,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus)
4,{'schema': 'https://schemas.data.amsterdam.nl/...,919937,3.630980e+12,"{'type': 'Point', 'coordinates': [120154.43, 4...",Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,1985.0,Quercus robur,NaN,NaN,Boom niet vrij uitgroeiend,Bomen,Quercus,Eik (Quercus)


In [13]:
# Assuming your DataFrame is named 'tree' and has a 'geometrie' column
# Replace 'geometrie' with the actual column name in your DataFrame

# Safely evaluate the 'geometrie' column as a literal
tree["geometrie"] = tree["geometrie"].apply(literal_eval)

# Extract RD coordinates from the 'geometrie' column
tree["RD_X"] = tree["geometrie"].apply(lambda x: x["coordinates"][0])
tree["RD_Y"] = tree["geometrie"].apply(lambda x: x["coordinates"][1])

# Convert RD coordinates to integers (optional)
tree["RD_X"] = tree["RD_X"].astype(int)
tree["RD_Y"] = tree["RD_Y"].astype(int)

# Display the extracted RD coordinates
print(tree[["RD_X", "RD_Y"]])

          RD_X    RD_Y
0       122115  483653
1       121980  483685
2       118737  482805
3       122053  484657
4       120154  483804
...        ...     ...
285879  118161  484418
285880  118199  484248
285881  117872  484305
285882  120740  484090
285883  120688  484037

[285884 rows x 2 columns]


In [14]:
# Define the RD coordinate system
rd = Proj(init="epsg:28992", preserve_units=False)

# Define the WGS84 coordinate system
wgs84 = Proj(init="epsg:4326")

# Convert RD coordinates to longitude and latitude
tree["Longitude"], tree["Latitude"] = transform(
    rd, wgs84, tree["RD_X"].values, tree["RD_Y"].values
)

# Display the updated DataFrame with RD and WGS84 coordinates
print(tree[["RD_X", "RD_Y", "Longitude", "Latitude"]])

/root/.local/share/pipx/venvs/poetry/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/root/.local/share/pipx/venvs/poetry/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_1189/3748122145.py:8: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

          RD_X    RD_Y  Longitude   Latitude
0       122115  483653   4.904669  52.339813
1       121980  483685   4.902685  52.340093
2       118737  482805   4.855196  52.331980
3       122053  484657   4.903661  52.348833
4       120154  483804   4.875880  52.341050
...        ...     ...        ...        ...
285879  118161  484418   4.846569  52.346438
285880  118199  484248   4.847146  52.344913
285881  117872  484305   4.842341  52.345403
285882  120740  484090   4.884450  52.343657
285883  120688  484037   4.883692  52.343177

[285884 rows x 4 columns]


In [15]:
tree.head()

,_links,id,gbdBuurtId,geometrie,typeBeheerderPlus,boomhoogteklasseActueel,typeEigenaarPlus,jaarVanAanleg,soortnaam,stamdiameterklasse,standplaatsGedetailleerd,typeObject,typeSoortnaam,soortnaamKort,soortnaamTop,geometry,RD_X,RD_Y,Longitude,Latitude
0,{'schema': 'https://schemas.data.amsterdam.nl/...,919933,3.630980e+12,"{'type': 'Point', 'coordinates': [122115.11, 4...",Stadsdeel Zuid,e. 15 tot 18 m.,Gemeente Amsterdam,1948.0,Tilia americana,NaN,NaN,Boom niet vrij uitgroeiend,Bomen,Tilia,Linde (Tilia),"{'type': 'Point', 'coordinates': [122115.11, 4...",122115,483653,4.904669,52.339813
1,{'schema': 'https://schemas.data.amsterdam.nl/...,919934,3.630980e+12,"{'type': 'Point', 'coordinates': [121980.46, 4...",Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,1978.0,Ulmus hollandica 'Vegeta',NaN,Tegels,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),"{'type': 'Point', 'coordinates': [121980.46, 4...",121980,483685,4.902685,52.340093
2,{'schema': 'https://schemas.data.amsterdam.nl/...,919935,3.630980e+12,"{'type': 'Point', 'coordinates': [118737.81, 4...",Stadsdeel Zuid,c. 9 tot 12 m.,Gemeente Amsterdam,1990.0,Fraxinus excelsior 'Westhof's Glorie',"0,2 tot 0,3 m.",NaN,Boom niet vrij uitgroeiend,Bomen,Fraxinus,Es (Fraxinus),"{'type': 'Point', 'coordinates': [118737.81, 4...",118737,482805,4.855196,52.331980
3,{'schema': 'https://schemas.data.amsterdam.nl/...,919936,3.630980e+12,"{'type': 'Point', 'coordinates': [122053.62, 4...",Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,2002.0,Ulmus glabra 'Lutescens',NaN,NaN,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),"{'type': 'Point', 'coordinates': [122053.62, 4...",122053,484657,4.903661,52.348833
4,{'schema': 'https://schemas.data.amsterdam.nl/...,919937,3.630980e+12,"{'type': 'Point', 'coordinates': [120154.43, 4...",Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,1985.0,Quercus robur,NaN,NaN,Boom niet vrij uitgroeiend,Bomen,Quercus,Eik (Quercus),"{'type': 'Point', 'coordinates': [120154.43, 4...",120154,483804,4.875880,52.341050


In [18]:
# Export to a CSV file
tree[["id", "Longitude", "Latitude"]].to_csv("tree.csv", index=False)